In [3]:
import sys
import pathlib

import pandas as pd
import pandana as pdna
import geopandas as gpd

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

DIR = pathlib.Path('..')
sys.path.append(str(DIR))

DATA_DIR = DIR/'data/'

%matplotlib inline

In [4]:
nodes, edges = ua.osm.load.ua_network_from_bbox(
    bbox=(173.9053,-36.1068,175.3171,-37.3090),
)
ua.osm.network.create_osm_net(
    osm_edges=edges,
    osm_nodes=nodes,
    travel_speed_mph=3,
)

Requesting network data within bounding box from Overpass API in 9 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](-37.31080435,173.90530000,-36.89890382,174.35734505);>;);out;'}"
Downloaded 0.3KB from www.overpass-api.de in 0.75 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](-36.90982354,173.90544675,-36.48878337,174.37135010);>;);out;'}"
Downloaded 373.8KB from www.overpass-api.de in 1.31 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

In [5]:
loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(
    gtfsfeed_path=str((DATA_DIR/'Auckland_GTFS_20180305').resolve()),
)
ua.gtfs.network.create_transit_net(
    gtfsfeeds_dfs=loaded_feeds,
    day='thursday',
    timerange=['07:00:00', '19:00:00'],
)
ua.gtfs.headways.headways(
    gtfsfeeds_df=loaded_feeds,
    headway_timerange=['07:00:00','19:00:00'],
)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 27.62 seconds
--------------------------------
Processing GTFS feed: Auckland_GTFS_20180305
GTFS feed: Auckland_GTFS_20180305, stops uses the same stop_id across multiple agency_ids. This feed stops table will be modified from its original format to provide stop_ids for each agency using a one to many join
agency.txt agency_name column has more than one agency name listed. Unique agency id was assigned using the agency id and associated agency name.
Unique agency id operation complete. Took 13.85 seconds
Unique GTFS feed id operation complete. Took 0.01 seconds
Appended route type to stops
Appended route type to stop_times
--------------------------------
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 7.27 seconds
1 GTFS feed file(s) successfully read as dataframes:


In [6]:
urbanaccess_net = ua.network.ua_network
ua.network.integrate_network(
    urbanaccess_network=urbanaccess_net,
    headways=True,
    urbanaccess_gtfsfeeds_df=loaded_feeds,
    headway_statistic='mean',
)

Loaded UrbanAccess network components comprised of:
     Transit: 18,148 nodes and 586,291 edges;
     OSM: 130,791 nodes and 348,076 edges
routes successfully joined to transit nodes. Took 1.38 seconds
Connector edges between the OSM and transit network nodes successfully completed. Took 35.62 seconds
mean route stop headway will be used for pedestrian to transit edge impedance.
Headway impedance calculation completed. Took 0.82 seconds
Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 20.64 seconds
Network edge and node network integration completed successfully resulting in a total of 190,400 nodes and 1,600,206 edges:
     Transit: 59,609 nodes 586,291 edges;
     OSM: 130,791 nodes 348,076 edges; and
     OSM/Transit connector: 665,839 edges.


In [7]:
ua.network.save_network(
    urbanaccess_network=urbanaccess_net,
    filename='final_net_March2018.h5',
    dir=str(DATA_DIR.resolve()),
    overwrite_key = True,
)

New final_net_March2018.h5 hdf5 store created in dir: G:\My Drive\Company\SE0070_MOT_PT_Justice\data


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\tables\leaf.py:353: RuntimeWarning: overflow encountered in long_scalars
  expected_mb = (expectedrows * rowsize) // MB


edges saved in G:\My Drive\Company\SE0070_MOT_PT_Justice\data/final_net_March2018.h5 hdf5 store.
Using existing G:\My Drive\Company\SE0070_MOT_PT_Justice\data/final_net_March2018.h5 hdf5 store.
nodes saved in G:\My Drive\Company\SE0070_MOT_PT_Justice\data/final_net_March2018.h5 hdf5 store.


In [8]:
transit_ped_net = pdna.Network(
    urbanaccess_net.net_nodes["x"],
    urbanaccess_net.net_nodes["y"],
    urbanaccess_net.net_edges["from_int"],
    urbanaccess_net.net_edges["to_int"],
    urbanaccess_net.net_edges[["weight"]], 
    twoway=False,
)

In [9]:
census = gpd.read_file(
    GIS_DIR/'census_2018_MB_WGS84.geojson,
)
census['centroid'] = census.centroid
census['x'] = census.centroid.map(lambda x: x.x)
census['y'] = census.centroid.map(lambda y: y.y)

census['node_id'] = transit_ped_net.get_node_ids(census['x'], census['y'])

transit_ped_net.set(census.node_id, variable = census['Place_of_Work_2018'], name='jobs')
jobs_45 = transit_ped_net.aggregate(45, type='sum', decay='linear', name='jobs')
jobs_30 = transit_ped_net.aggregate(30, type='sum', decay='linear', name='jobs')
jobs_15 = transit_ped_net.aggregate(15, type='sum', decay='linear', name='jobs')


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  import sys
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [10]:
df = jobs_45.to_frame()
df.columns = ['jobs']
df['x']=transit_ped_net.nodes_df.x
df['y']=transit_ped_net.nodes_df.y
df

jobs           x          y
id_int                                      
1         1703.327077  175.018770 -36.784850
2         1649.418120  175.021950 -36.784440
3         1669.803672  175.026970 -36.783430
4         1724.052869  175.030210 -36.784160
5         1784.568536  175.033790 -36.783010
...               ...         ...        ...
190396   82164.419260  174.741235 -36.758069
190397   82395.333480  174.741129 -36.758032
190398  121328.600756  174.779586 -36.848869
190399  120690.330357  174.779582 -36.848800
190400  120907.354676  174.779583 -36.848824

[190400 rows x 3 columns]

In [11]:
df.to_csv(DATA_DIR/'jobs_MB2018.csv')